# Testing Individual components of the FV HE scheme

In [2]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [4]:
# poly_modulus = 8
# bit_sizes= [10]
# plain_modulus = 8
poly_modulus = 256
bit_sizes= [20]
plain_modulus = 256
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [6]:
print(ctx.param.coeff_modulus)

[1047041]


In [8]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[1047040, 1, 1, 1047040, 1, 1047040, 1047040, 0, 0, 1, 1047040, 1, 0, 0, 0, 1047040, 1, 0, 1, 1, 1, 0, 1, 1047040, 1047040, 1, 1047040, 1, 0, 1047040, 0, 1047040, 0, 1047040, 1, 0, 1047040, 1, 1047040, 1, 1047040, 1047040, 1047040, 1047040, 0, 0, 1, 1, 1, 0, 1047040, 1047040, 1047040, 1047040, 1, 1047040, 1, 0, 0, 1, 1, 1047040, 1047040, 1, 1, 0, 1047040, 1, 0, 1047040, 1047040, 1, 1, 1047040, 0, 1, 0, 0, 1047040, 1, 1, 0, 1047040, 1, 1047040, 1, 1, 0, 0, 1, 1, 1, 0, 1047040, 1, 0, 1047040, 0, 0, 1047040, 1, 1047040, 1047040, 1047040, 1047040, 1047040, 1, 0, 0, 0, 0, 1047040, 1, 1, 0, 1047040, 0, 1047040, 0, 1047040, 0, 1, 1, 1, 1047040, 0, 1047040, 1047040, 1, 1, 0, 1047040, 1, 1, 1, 0, 0, 1047040, 1, 0, 1, 1047040, 1, 1, 1047040, 1, 1047040, 1047040, 1, 0, 1047040, 1047040, 0, 1, 1047040, 1047040, 0, 1, 1, 1, 0, 1, 0, 1047040, 1, 1, 0, 1047040, 0, 1047040, 0, 1, 1047040, 1, 1047040, 0, 0, 0, 0, 0, 0, 0, 1047040, 1047040, 1047040, 0, 0, 1, 1047040, 1047040, 10470

In [10]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [12]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[0, 0, 0, 1]     [0, 0, 1]     [1]


### Decodes back to Integer

In [14]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

8
4
1


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [16]:
encrypter = Encryptor(ctx, pk)

In [18]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [20]:
decrypter = Decryptor(ctx, sk)

In [22]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [24]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

8     4     1


## Evaluator

In [26]:
eval = Evaluator(ctx)

In [28]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

12


In [30]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

12


In [32]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

12


### Verify result

In [34]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [36]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)

result : [[[181683, 617010, 1029873, 856260, 385879, 440583, 560731, 576224, 839281, 160634, 616707, 955084, 763983, 459952, 167056, 219179, 512548, 797697, 322149, 139298, 989771, 885416, 369843, 471339, 548275, 365631, 659799, 133187, 502995, 617286, 862557, 463495, 930391, 944507, 208353, 79551, 557682, 767844, 1025926, 668962, 805927, 705293, 13087, 815630, 881698, 1031203, 685780, 335155, 475728, 963254, 1000291, 863448, 905843, 889975, 1012538, 367946, 91578, 488472, 871557, 340307, 940758, 992815, 908473, 339004, 745013, 1039923, 113612, 491742, 624343, 670911, 543971, 582954, 861087, 359569, 673211, 290980, 144025, 25727, 531094, 661841, 941093, 482252, 592995, 5221, 308997, 284298, 188619, 1014195, 892639, 703509, 72595, 418231, 516015, 438465, 836592, 834911, 433489, 260573, 339832, 659825, 1038974, 289529, 1038810, 202574, 569540, 594332, 1009934, 719260, 60665, 905095, 240805, 398218, 423156, 867403, 881338, 331132, 868636, 198918, 152816, 431315, 925190, 715673, 542200, 60

In [38]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

32      -2375857022819183889872706970761224772651735965315504612512865606180936511704460


AssertionError: 